In [1]:
import os 
import sys

sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, Conv2D, MaxPooling2D, Dropout, LSTM, Bidirectional, Flatten 
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_curve



Using TensorFlow backend.


In [2]:
x_train = np.load(os.path.join('../train_data_proc','x_0.npy'))
y_train = np.load(os.path.join('../train_data_proc', 'y_0.npy'))

x_test = np.load(os.path.join('../test_data_proc','x_0.npy'))
y_test = np.load(os.path.join('../test_data_proc', 'y_0.npy'))

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(11506, 88)
(11500, 88)
(4806, 88)
(4800, 88)


In [3]:
context_window_size = 7
frequencies_size = x_train.shape[1]
sequence_size = 100
batch_size = 1

output_shape = y_train.shape[1]

In [4]:
keras.backend.clear_session()

In [5]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'), \
           input_shape=(sequence_size, context_window_size,frequencies_size,1)
    )
)
model.add(Dropout(0.5))
model.add(TimeDistributed(MaxPooling2D((1,3), strides=(1,1))))
          
model.add(TimeDistributed(Conv2D(128, (1,3), activation='relu')))
model.add(Dropout(0.5))
model.add(TimeDistributed(MaxPooling2D((1,3), strides=(1,1))))

model.add(TimeDistributed(Flatten()))

model.add(TimeDistributed(Dense(200)))
model.add(Dropout(0.5))

model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.5), merge_mode ='ave'))

model.add(TimeDistributed(Dense(output_shape, activation='sigmoid')))
model.compile(loss='categorical_crossentropy', optimizer='adam')
          


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
inp = model.input                                           # input placeholder
outputs = [(layer.input,layer.output) for layer in model.layers]          # all layer outputs
print ('\n'.join(map(str,outputs)))

(<tf.Tensor 'time_distributed_1_input:0' shape=(?, 100, 7, 88, 1) dtype=float32>, <tf.Tensor 'time_distributed_1/Reshape_1:0' shape=(?, 100, 5, 86, 64) dtype=float32>)
(<tf.Tensor 'time_distributed_1/Reshape_1:0' shape=(?, 100, 5, 86, 64) dtype=float32>, <tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 100, 5, 86, 64) dtype=float32>)
(<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 100, 5, 86, 64) dtype=float32>, <tf.Tensor 'time_distributed_2/Reshape_1:0' shape=(?, 100, 5, 84, 64) dtype=float32>)
(<tf.Tensor 'time_distributed_2/Reshape_1:0' shape=(?, 100, 5, 84, 64) dtype=float32>, <tf.Tensor 'time_distributed_3/Reshape_1:0' shape=(?, 100, 5, 82, 128) dtype=float32>)
(<tf.Tensor 'time_distributed_3/Reshape_1:0' shape=(?, 100, 5, 82, 128) dtype=float32>, <tf.Tensor 'dropout_2/cond/Merge:0' shape=(?, 100, 5, 82, 128) dtype=float32>)
(<tf.Tensor 'dropout_2/cond/Merge:0' shape=(?, 100, 5, 82, 128) dtype=float32>, <tf.Tensor 'time_distributed_4/Reshape_1:0' shape=(?, 100, 5, 80, 128) dtype=floa

In [7]:
import datagen

In [8]:
import importlib
datagen = importlib.reload(datagen)

In [9]:
data_generator = datagen.DataGenerator('./train_data_proc/', 100,256, 7, 'x_[0-9]+.npy', 'y_[0-9]+.npy')

11500
now listing inputs
['x_0.npy']
now listing outputs
['y_0.npy']


In [10]:
print(data_generator)


In [11]:
model.fit_generator(generator=data_generator.__getitem__(),
                    use_multiprocessing=False,
                     epochs=20, steps_per_epoch=data_generator.__len__())

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
starting
y_0.npy
104/115 [==========================>...] - ETA: 6s - loss: 7.9998one up
y_0.npy
115/115 [==============================] - 69s 600ms/step - loss: 8.0448
Epoch 2/20
104/115 [==========================>...] - ETA: 5s - loss: 7.2973one up
y_0.npy
115/115 [==============================] - 61s 530ms/step - loss: 7.3820
Epoch 3/20
104/115 [==========================>...] - ETA: 5s - loss: 7.0934one up
y_0.npy
115/115 [==============================] - 61s 529ms/step - loss: 7.1735
Epoch 4/20
104/115 [==========================>...] - ETA: 5s - loss: 6.8879one up
y_0.npy
115/115 [==============================] - 61s 527ms/step - loss: 6.9644
Epoch 5/20
104/115 [==========================>...] - ETA: 5s - loss: 6.6966one up
y_0.npy
115/115 [==============================] - 61s 529ms/step - loss: 6.7630
Epoch 6/20
104/115 [==========================>...] - ETA: 5s - loss: 6.4410one up
y_0.npy
115/115 [===============

In [12]:
test_generator= datagen.DataGenerator('../test_data_proc/', 100,256, 7, 'x_[0-9]+.npy', 'y_[0-9]+.npy')


4800
now listing inputs
['x_0.npy']
now listing outputs
['y_0.npy']


In [13]:
print(test_generator.__len__())
predictions = model.predict_generator(generator=test_generator.__getitemtest__(),steps=test_generator.__len__())

48
y_0.npy
one up


In [14]:
print(predictions.shape)

(48, 100, 88)


In [15]:
predictions = np.reshape(predictions, (predictions.shape[0]*predictions.shape[1], predictions.shape[2]))

In [16]:
predictions.shape

(4800, 88)

In [17]:
labels = np.array([item for item in test_generator.__getitemtestlabels__()])[0]

In [18]:
labels.shape

(4800, 88)

In [19]:
model.save('shitty_LSTM')

In [20]:
data_generator.__len__()

115